## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importando o dataset

In [2]:
data = pd.read_csv('adm_data.csv')
data.rename(columns = {'Chance of Admit ': 'Chance of Admit'}, inplace = True)
data.rename(columns = {'LOR ': 'LOR'}, inplace = True)

In [3]:
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [4]:
features = ['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research']
label = 'Chance of Admit'

### Correlação dos dados com a chance de admissão

In [5]:
data.corr()['Chance of Admit']

Serial No.           0.042336
GRE Score            0.802610
TOEFL Score          0.791594
University Rating    0.711250
SOP                  0.675732
LOR                  0.669889
CGPA                 0.873289
Research             0.553202
Chance of Admit      1.000000
Name: Chance of Admit, dtype: float64

Temos, de fato, dados altamente correlacionados com a chance de admissão dos estudantes nas universidades escolhidas.

## Modelo de Regressão Linear Anteriormente Implementado

In [6]:
class LinearRegressionModel():
    def __init__(self, features, label, ws: list,
                 b = 0, alpha = 0.1, random_state = 0):
        '''Constructor for the Multi_Features_LinearRegressionModel class. It takes in seven arguments:

        - features: a Pandas DataFrame containing the features to be used for training the model;
        - label: a Pandas Series containing the label corresponding to the features that will be predicted using the model;
        - ws: a list of floats representing the initial values of the weights (coefficients) of the features;
        - b: a float representing the initial value of the bias term (default value is 0);
        - alpha: a float representing the learning rate (i.e., the size of the step taken in the direction of the gradient during gradient descent) (default value is 0.1);
        - random_state: an integer representing the random seed to use for generating random numbers (default value is 0).'''
        self.features = features
        self.label = label
        ws = [float(w) for w in ws]
        self.ws = np.array(ws)
        self.b = b
        self.alpha = alpha
        self.rand = np.random.RandomState(random_state)
        
    def print_parameters(self):
        '''Prints the current values of the weights ws and bias b of the model.'''
        for i in range(1, len(self.ws) + 1):
            print(f'w{i} = {self.ws[i - 1]}')
        print (f'b = {self.b}')
    
    def predict(self, X):
        '''Takes in a Pandas DataFrame X containing feature values 
        and returns a NumPy array of predicted values of the label using 
        the current values of ws and b.'''
        predictions = np.array(X.mul(self.ws).sum(axis = 1) + self.b)
        return predictions
    
    def get_loss(self, X, y):
        '''Takes in a Pandas DataFrame X containing feature values 
        and a Pandas Series y containing the corresponding true label values, 
        and returns the mean squared error loss of the model on the data.'''
        predictions = self.predict(X)
        diff = predictions - y
        loss = np.mean(diff**2)
        return loss
    
    def __get_Xy_sample(self, begin_index, end_index):
        '''Private helper function which from X and y starting in the begin_index 
        row and ending in end_index row.'''
        X = self.features.iloc[begin_index:end_index]
        y = self.label.iloc[begin_index:end_index]
        return X, y
    
    def __get_sample_partial_w(self, X, diff, batch_size):
        '''Private helper function which gets the partial derivative of loss with 
        respect to weights for a sample (X).'''
        partial_w = (2/batch_size) * (diff @ X)
        return partial_w
    
    def __get_sample_partial_b(self, diff, batch_size):
        partial_b = (2/batch_size) * np.sum(diff)
        return partial_b
    
    def __batch_update_parameters(self, X, diff, batch_size, inexact_batch_size):
        '''Private helper function which updates weights (ws) and bias (b) for a batch.'''
        partial_w = self.__get_sample_partial_w(X, diff, inexact_batch_size)
        partial_b = self.__get_sample_partial_b(diff, inexact_batch_size)
        correction_constant = batch_size/inexact_batch_size
        self.ws -= self.alpha * partial_w * correction_constant
        self.b -= self.alpha * partial_b * correction_constant
        
    def __sgd_update_parameters(self, batch_size: int):
        '''Private helper function that performs one step of stochastic gradient descent on the model's parameters (ws and b). 
        It takes in a single argument batch_size, which is the number of samples to use in the mini-batch for this step of gradient descent. 
        The function first selects mini-batches from the training data, and then performs an update to the model's parameters 
        using the gradient of the mean squared error loss with respect to the parameters for each mini-batch.
        The update is performed using the learning rate alpha.'''
        num_of_data_rows = len(self.label)
        inexact_batch_size = num_of_data_rows % batch_size
        num_of_exact_batches = int(num_of_data_rows/batch_size)
        for exact_batch in range(1, num_of_exact_batches + 1):
            begin_index = (exact_batch - 1) * batch_size
            end_index = exact_batch * batch_size
            X, y = self.__get_Xy_sample(begin_index, end_index)
            predictions = self.predict(X)
            diff = predictions - y
            self.__batch_update_parameters(X, diff, batch_size, batch_size)
        if inexact_batch_size != 0:
            begin_index = (num_of_exact_batches) * batch_size + 1
            end_index = num_of_data_rows + 1
            X, y = self.__get_Xy_sample(begin_index, end_index)
            predictions = self.predict(X)
            diff = predictions - y
            self.__batch_update_parameters(X, diff, batch_size, inexact_batch_size)
        
    def sgd(self, iterations: int, batch_size: float, print_loss: bool): # stochastic gradient descent
        '''Performs stochastic gradient descent for a specified number of iterations. It takes in three arguments:

        - iterations: an integer representing the number of iterations of gradient descent to perform
        - batch_size: a float representing the number of samples to use in each mini-batch for each step of gradient descent
        - print_loss: a boolean indicating whether to print the loss after each iteration of gradient descent. 
        If True, the loss will be printed; if False, the loss will not be printed.'''
        for i in range(0, iterations):
            self.__sgd_update_parameters(batch_size)
            if print_loss:
                print(f'loss = {self.get_loss(self.features, self.label)}')
    
    @staticmethod
    def shuffle_data(data, random_state):
        '''Shuffles a Pandas Dataframe's data.'''
        rand = np.random.RandomState(random_state)
        return data.reindex(rand.permutation(data.index))
    
    @staticmethod
    def train_val_test_split(X, y, test_split_factor: float, val_split_factor: float):
        '''Get train, validation and test Pandas Dataframes from X (features) and y (label).'''
        num_of_data_rows = len(y)
        test_size = int(test_split_factor * num_of_data_rows)
        val_size = int(val_split_factor * num_of_data_rows)
        X_test = X.iloc[0:test_size]
        y_test = y.iloc[0:test_size]
        X_val = X.iloc[test_size:test_size + val_size]
        y_val = y.iloc[test_size:test_size + val_size]
        X_train = X.iloc[test_size + val_size:num_of_data_rows]
        y_train = y.iloc[test_size + val_size:num_of_data_rows]
        return X_test, y_test, X_val, y_val, X_train, y_train

## Treinamento, Validação e Teste

Vamos treinar o modelo no dataset que temos. Para tal, iremos dividir o dataframe 'data' em três partes: training, validation e test. A primeira parte será utilizada para o treinamento do modelo; a segunda, para validar o modelo em um conjunto de dados diferentes daqueles usados no treinamento e, se necessário, para corrigir os parâmetros do modelo; e a terceira será utilizada como uma forma de teste final para o modelo, evitando o overfitting nos dados de validação.

### Função para separar os dados em treinamento, validação e teste

Tal função está implementada dentro da classe LinearRegressionModel, juntamente da função shuffle_data, utilizada para embaralhar os dados no dataframe.

#### Shuffling

In [7]:
random_state = 0
data_shuffled = LinearRegressionModel.shuffle_data(data, random_state = random_state)

#### Split

In [8]:
X = data_shuffled[features]
y = data_shuffled[label]

In [9]:
X_test, y_test, X_val, y_val, X_train, y_train = LinearRegressionModel.train_val_test_split(X, y, test_split_factor = 0.2, val_split_factor = 0.2)

In [10]:
X_train.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
281,317,110,3,4.0,4.5,9.11,1
133,323,112,5,4.0,4.5,8.78,0
33,340,114,5,4.0,4.0,9.60,1
378,303,98,1,2.0,2.5,7.65,0
162,318,109,3,3.0,3.0,8.50,0


In [11]:
X_train.shape

(240, 7)

In [12]:
X_val.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
336,319,110,3,3.0,2.5,8.79,0
64,325,111,3,3.0,3.5,8.70,0
55,320,103,3,3.0,3.0,7.70,0
106,329,111,4,4.5,4.5,9.18,1
300,309,106,2,2.5,2.5,8.00,0


In [13]:
X_val.shape

(80, 7)

In [14]:
X_test.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
132,309,105,5,3.5,3.5,8.56,0
309,308,110,4,3.5,3.0,8.60,0
341,326,110,3,3.5,3.5,8.76,1
196,306,105,2,3.0,2.5,8.26,0
246,316,105,3,3.0,3.5,8.73,0


In [15]:
X_test.shape

(80, 7)

# Pre-processing

#### Obs: o pré-processamento deve vir depois do processo de split, para evitar adicionar viés aos dados de teste por influência dos dados de treino.

### Mudando a escala dos dados para [0, 1]

Transformaremos [min, max] -> [0, 1] para cada coluna.

In [16]:
def scale(feature, train_feature):
    minimum = min(train_feature)
    maximum = max(train_feature)
    return (feature - minimum)/(maximum - minimum)

In [17]:
unscaled_X_train= X_train.copy()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()
for feature in features:
    X_train_scaled[feature] = scale(X_train_scaled[feature], unscaled_X_train[feature])
    X_val_scaled[feature] = scale(X_val_scaled[feature], unscaled_X_train[feature])
    X_test_scaled[feature] = scale(X_test_scaled[feature], unscaled_X_train[feature])

### Treinando o modelo

In [18]:
ws = list(np.zeros(len(features)))
b = 0
alpha = 0.01

In [19]:
model = LinearRegressionModel(features = X_train_scaled,
                                label = y_train, ws = ws, b = b, alpha = alpha,
                                random_state = random_state)

In [20]:
model.get_loss(X_train_scaled, y_train)

0.5472704166666664

In [21]:
model.sgd(iterations = 20, batch_size = 20, print_loss = False)

In [22]:
model.get_loss(X_train_scaled, y_train)

0.006264075583736085

### Aplicando o modelo nos dados de validação

In [23]:
model.predict(X_val_scaled)

336    0.655702
64     0.703057
55     0.597302
106    0.890603
300    0.544574
         ...   
136    0.702269
275    0.759861
126    0.804892
198    0.705632
299    0.658937
Length: 80, dtype: float64

In [24]:
model.get_loss(X_val_scaled, y_val)

0.006077148662474384

As perdas relativas à train_data e à val_data estão muito próximas e estão suficientemente baixas. Logo, falta somente conferirmos se o modelo não está overfitted por meio test_data.

### Aplicando o modelo nos dados de teste

In [25]:
model.predict(X_test_scaled)

132    0.693544
309    0.675136
341    0.776928
196    0.559177
246    0.658413
         ...   
14     0.642589
363    0.557520
304    0.556071
361    0.891407
329    0.436774
Length: 80, dtype: float64

In [26]:
model.get_loss(X_test_scaled, y_test)

0.006625737302429019

O resultado está bem coerente e configura uma possível predição. Assim, conseguimos treinar o modelo, validá-lo e testá-lo com sucesso!

### Parâmetros finais do modelo

In [27]:
model.print_parameters()

w1 = 0.11530799078359782
w2 = 0.1176373524513257
w3 = 0.08892145823187657
w4 = 0.11431736723098168
w5 = 0.1331000425434934
w6 = 0.13720430956388835
w7 = 0.05883796135278343
b = 0.27415540401434535
